# Bayesian Search Algorithm

Use bayesian search based on the paper: "Bayesian-Based Search Decision Framework and Search Strategy Analysis in Probabilistic Search"

link: https://onlinelibrary.wiley.com/doi/full/10.1155/2020/8865381

In [568]:
# Import necessary libraries
import networkx as nx
import random

In [569]:
# Constants
map_graph_path = "../graphs/outputs/stricted_graph.graphml" # .graphml file path
start_node = "n200"   # Start position (node)
target_node = "n220" # target's (ring) position (node)
p_tp = 0.8  # P(true positive)
p_fp = 0.2  # P(false positive)
p_tn = 0.2  # P(true negative)
p_fn = 0.8  # P(false negative)
B_lower = 0  # Lower boundary for maximum belief
B_upper = 0.99   # Upper boundary for maximum belief
max_steps = 500  # Maximum number of steps

# May I say "Fuck this paper's notations. What the fuck do you mean by 3 same equations having 3 distinct values??? What the hell is this level of inconsistency in naming variables???."

In [570]:
# Generic helper functions

# Display graph information
def display_grpah_information(graph):
    # Print nodes data
    for node, data in graph.nodes(data=True):
        print(f"Node: {node}, Attributes: {data}")

    # Print edges data
    for u, v, data in graph.edges(data=True):
        print(f"Edge: ({u}, {v}), Attributes: {data}")

    print("Number of Nodes:", len(graph.nodes()))
    print("Number of Edges:", len(graph.edges()))

def add_edges_weight(graph):
    for u, v, data in graph.edges(data=True):
        data['weight'] = 1.0  # Assign a weight of 1 to each edge

In [571]:
# Set target (ring) position (node)
def set_target_node(graph, target_node_id):
    nx.set_node_attributes(graph, {target_node_id: {"target": True}})

In [572]:
# Bayesian search for incomplete information sensor

# Create an initial belief map
def initialize_belief_map(graph):
    num_nodes = len(graph.nodes())
    uniform_belief = 1.0 / num_nodes

    for node in graph.nodes():
        graph.nodes[node]["P(target)"] = uniform_belief


# Implement the Dijkstra algorithm
def dijkstra_search(graph, start_node, target_node):
    try:
        path = nx.dijkstra_path(graph, source=start_node, target=target_node, weight='weight')
        
        return path
    except nx.NetworkXNoPath:
        print(f"No path found between {start_node} and {target_node}.")
        return []

# Update the belief map
def update_belief_map(graph, agent_node, observation):
    total_nodes = len(graph.nodes())
    new_beliefs = {}
    
    # Calculate the denominator for the Bayesian update (marginal probability)
    denominator = 0.0

    for node in graph.nodes():
        prior_prob = graph.nodes[node]["P(target)"]

        if node == agent_node:
            if observation:
                denominator += prior_prob * p_tp  # P(O | D_d) * P(D_d)
            else:
                denominator += prior_prob * p_fn  # P(~O | D_d) * P(D_d)
        else:
            if observation:
                denominator += prior_prob * p_fp  # P(O | D_c) * P(D_c)
            else:
                denominator += prior_prob * p_tn  # P(~O | D_c) * P(D_c)

    # Quit if denominator = 0
    if denominator == 0:
        return

    # Updated probabilities
    new_beliefs = {}

    for node in graph.nodes():
        prior_prob = graph.nodes[node]["P(target)"]

        if node == agent_node:
            if observation:
                numerator = prior_prob * p_tp  # P(D_d | O)
            else:
                numerator = prior_prob * p_fn  # P(D_d | ~O)
        else:
            if observation:
                numerator = prior_prob * p_fp  # P(D_c | O)
            else:
                numerator = prior_prob * p_tn  # P(D_c | ~O)
                
        updated_prob = numerator / denominator
        new_beliefs[node] = updated_prob

    # Assign the new beliefs to the graph nodes
    nx.set_node_attributes(graph, new_beliefs, "P(target)")

# Modified saccadic search strategy
def saccadic_search_strategy(graph, start_node, max_steps):
    current_node = start_node
    path_taken = [current_node]
    target_found = False
    
    # Initialize with uniform belief
    initialize_belief_map(graph)

    current_path = []
    
    for step in range(max_steps):
        print(f"--- Step {step + 1} ---")
        
        # Check if we have reached our planned destination and need a new one
        if not current_path:
            # Find the node with the highest belief
            highest_belief_node = max(graph.nodes(data='P(target)'), key=lambda x: x[1])[0]
            
            print(f"New target destination identified: {highest_belief_node}.")
            
            # Plan a new path to the highest belief node
            current_path = dijkstra_search(graph, current_node, highest_belief_node)
            
            # If no path is found, break the loop
            if not current_path:
                print(f"No path found to {highest_belief_node}. Stopping search.")
                break
                
            # The first element is the current node, we want to move to the next.
            if len(current_path) > 1:
                current_path.pop(0)

        # Move to the next node on the path
        if current_path:
            next_node = current_path.pop(0)
            print(f"Moving from {current_node} to {next_node}...")
            current_node = next_node
            path_taken.append(current_node)
        
        # Perform observation and update belief at the new location
        is_target = graph.nodes[current_node].get("target", False)
        if is_target:
            observation = random.random() < p_tp   # True positive with probability p_tp
        else:
            observation = random.random() < p_fp   # False positive with probability p_fp
        
        update_belief_map(graph, current_node, observation)
        
        print(f"At {current_node}. Belief at this node is {graph.nodes[current_node]['P(target)']:.4f}")

        # Check for target presence. We check at every step because we're moving and observing.
        if is_target and observation:
             print(f"Target found at {current_node}!")
             target_found = True
             break
    
    return path_taken, target_found

In [573]:
# Read .graphml file
map_graph = nx.read_graphml(map_graph_path)

# display_grpah_information(map_graph)

In [574]:
# Run the search and get results
print("\nStarting Saccadic Search...")
search_path, found = saccadic_search_strategy(map_graph, start_node, max_steps)
print("\nSearch Results:")
if found:
    print("Target was found!")
else:
    print("Target was not found within the maximum number of steps.")
print("Search path:", search_path)


Starting Saccadic Search...
--- Step 1 ---
New target destination identified: n0.
Moving from n200 to n201...
At n201. Belief at this node is 0.0056
--- Step 2 ---
Moving from n201 to n38...
At n38. Belief at this node is 0.0055
--- Step 3 ---
Moving from n38 to n26...
At n26. Belief at this node is 0.0055
--- Step 4 ---
Moving from n26 to n25...
At n25. Belief at this node is 0.0055
--- Step 5 ---
Moving from n25 to n28...
At n28. Belief at this node is 0.0055
--- Step 6 ---
Moving from n28 to n41...
At n41. Belief at this node is 0.0055
--- Step 7 ---
Moving from n41 to n13...
At n13. Belief at this node is 0.0054
--- Step 8 ---
Moving from n13 to n1...
At n1. Belief at this node is 0.0054
--- Step 9 ---
Moving from n1 to n0...
At n0. Belief at this node is 0.0054
--- Step 10 ---
New target destination identified: n0.
Moving from n0 to n0...
At n0. Belief at this node is 0.0212
--- Step 11 ---
New target destination identified: n0.
Moving from n0 to n0...
At n0. Belief at this node 